# Telco Customer Churn Tahmini

Telco şirketinin müşterilerinin sistem içerisinde kalıp kalmayacaklarını tahmin eden bir makine öğrenmesi modeli oluşturulması amaçlanmıştır.


Gerekli kütüphanelerin yüklenmesi

In [2]:
import numpy as np #lineer cebir kütüphanesi
import pandas as pd # veri işleme
import seaborn as sns
import joblib
import missingno as msno

from matplotlib import pyplot as plt
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier

Dataseti yüklemek

In [4]:
data = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df=data.copy()
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.65,Yes


Veriyi İnceleme

In [8]:
def check_df(data): # Verinin genel detaylarını gösteren fonksiyon
    print('##################### shape ##########################')
    print(data.shape)
    print('##################### type ##########################')
    print(data.dtypes)
    print('##################### null ##########################')
    print(data.isnull().sum().sort_values(ascending=False))
    print('#################### quantiles #####################')
    print(data.describe([0, 0.05, 0.5, 0.95, 0.99, 1]).T)
check_df(df)

##################### shape ##########################
(7043, 21)
##################### type ##########################
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object
##################### null ##########################
customerID          0
DeviceProtection    0
TotalCharges        0
MonthlyCharges      0
PaymentMethod       0
PaperlessBilling    0
Contract            0
StreamingMovies     0
StreamingTV         0
TechSupport         0
Onli

Sonuç:

-21 özellik mevcut

-Çoğu kategorik veri

-Nan değere sahip bir özellik yok

-"SeniorCitizen", "tenure", "MonthlyCharges" nümerik tipte geri kalanı kategorik veri

In [10]:
#TotalCharges tipini string değerden sayısal değere geçirme
df['TotalCharges']= pd.to_numeric(df['TotalCharges'],errors = 'coerce')
df.isnull().sum()
print(df.shape)
# Tip değişiminden sonra 11 missing değer ortaya çıktı. 11 veri DROP yedi.
df = df[df['TotalCharges'].notna()]
print(df.shape)

(7043, 21)
(7032, 21)


In [11]:
# Kaç tane kategorik veri sayısal olarak değiştirebilir ya da tam tersi mümkün mü kontrolü
def grab_col_names (dataframe, categorical = 10, cardinal =20):

    # kategorik
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == 'O']
    cat_but_car = [col for col in dataframe.columns if dataframe[col].dtype == 'O' and
                   dataframe[col].nunique() > cardinal]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].dtype != 'O' and
                   dataframe[col].nunique() < categorical]
    cat_cols += num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # sayısal

    num_cols = [col for col in dataframe.columns if dataframe[col].dtype != 'O' and
                col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names (df, categorical = 10, cardinal =20)

print('Categorical Columns :',cat_cols)
print('Numerical Columns :',num_cols)
print('Numerical seemed categorical columns :',cat_but_car)

Observations: 7032
Variables: 21
cat_cols: 17
num_cols: 3
cat_but_car: 1
num_but_cat: 1
Categorical Columns : ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn', 'SeniorCitizen']
Numerical Columns : ['tenure', 'MonthlyCharges', 'TotalCharges']
Numerical seemed categorical columns : ['customerID']


Hedef Çalışması

In [12]:
# Sayısal edğişkenler ile hedef değişkenlerini karşılaştırma
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: 'mean'}))

for col in num_cols:
    target_summary_with_num(df, 'Churn', col)

# Tenure : Müşteri şirkette kaç ay kaldı.
# Churn : No: Müşteri şirkette kaldı. 
# Churn : Yes: Müşteri şirketten ayrıldı.
# Şirkette kalanlan yaklaşık 37 zaman geçirdi.
# Şirketten ayrılanlar 18 ay zaman geçirdi.
# MonthlyCharges: Aylık Ödeme
# Kalanlar aylık 61.2 ödedi
# Gidenler aylık 74.4 ödedi.

       tenure
Churn        
No     37.650
Yes    17.979
       MonthlyCharges
Churn                
No             61.307
Yes            74.441
       TotalCharges
Churn              
No         2555.344
Yes        1531.796


In [13]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.850,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,1889.500,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.150,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,1840.750,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.650,Yes


In [14]:
#Kategorik özellikler hedef değişkenle doğru şekilde karşılaştırılabilsin diye hedef değeri 'yes-no' formatından '1-0' formatına çevirme işlemi
df['Churn'] = df['Churn'].apply(lambda x: 1 if (x == 'Yes') else 0)
# Yes: Churned. Hizmeti satın almıyor. 1
# No: Not Churned. Hizmeti satın alıyor. 0

In [15]:
# Kategorik dğerleri hedef değer ile karşılaştrıma

def target_summary_with_cat(dataframe, target, categorical_col):
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean()}), end="\n\n\n")

for col in cat_cols:
    target_summary_with_cat(df, "Churn", col)

        TARGET_MEAN
gender             
Female        0.270
Male          0.262


         TARGET_MEAN
Partner             
No             0.330
Yes            0.197


            TARGET_MEAN
Dependents             
No                0.313
Yes               0.155


              TARGET_MEAN
PhoneService             
No                  0.250
Yes                 0.267


                  TARGET_MEAN
MultipleLines                
No                      0.251
No phone service        0.250
Yes                     0.286


                 TARGET_MEAN
InternetService             
DSL                    0.190
Fiber optic            0.419
No                     0.074


                     TARGET_MEAN
OnlineSecurity                  
No                         0.418
No internet service        0.074
Yes                        0.146


                     TARGET_MEAN
OnlineBackup                    
No                         0.399
No internet service        0.074
Yes                        0.2

# Özellik / Sütün Yapılandırma


In [16]:
df.head()

#Her CustomerId unique. Analize bir katkısı yok o yüzden drop 
df.drop('customerID', axis = 1, inplace = True)

In [17]:
for col in num_cols:
    target_summary_with_num(df, 'Churn', col)

df['TENURE_CAT_NEW'] = pd.cut(x= df['tenure'], bins = [-1, 1, 3, 6, 12, 24, 32, 75],
                              labels = [ 'Very_short', 'Short', 'medium1','medium2', 'medium3' , 'long1', 'long2'])
df['MONTHLY_CHARGES_NEW'] = pd.cut(x = df['MonthlyCharges'], bins = [-1, 20, 30, 55, 65, 75, 110],
                                  labels = [ 'economy', 'upper_eco', '2x_upper_eco', 'premium' , 'platinium', 'x_platinum'])
df.info()

       tenure
Churn        
0      37.650
1      17.979
       MonthlyCharges
Churn                
0              61.307
1              74.441
       TotalCharges
Churn              
0          2555.344
1          1531.796
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   gender               7032 non-null   object  
 1   SeniorCitizen        7032 non-null   int64   
 2   Partner              7032 non-null   object  
 3   Dependents           7032 non-null   object  
 4   tenure               7032 non-null   int64   
 5   PhoneService         7032 non-null   object  
 6   MultipleLines        7032 non-null   object  
 7   InternetService      7032 non-null   object  
 8   OnlineSecurity       7032 non-null   object  
 9   OnlineBackup         7032 non-null   object  
 10  DeviceProtection     7032 non-null   object  
 11  T

In [19]:
df['Contract_Extra_Services'] = (df[['PhoneService', 'InternetService', 'OnlineSecurity',
                                       'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                       'StreamingTV', 'StreamingMovies']]== 'Yes').sum(axis=1)

In [20]:
# Kişinin toplam aldığı servis sayısı
df['NEW_TotalServices'] = (df[['PhoneService', 'InternetService', 'OnlineSecurity',
                                       'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                       'StreamingTV', 'StreamingMovies']]== 'Yes').sum(axis=1)


# Kontratı 1 veya 2 yıllık müşterileri Engaged olarak belirtme
df["NEW_Engaged"] = df["Contract"].apply(lambda x: 1 if x in ["One year","Two year"] else 0)

# Herhangi bir destek, yedek veya koruma almayan kişiler
df["NEW_noProt"] = df.apply(lambda x: 1 if (x["OnlineBackup"] != "Yes") or (x["DeviceProtection"] != "Yes") or (x["TechSupport"] != "Yes") else 0, axis=1)

# Aylık sözleşmesi bulunan ve genç olan müşteriler
df["NEW_Young_Not_Engaged"] = df.apply(lambda x: 1 if (x["NEW_Engaged"] == 0) and (x["SeniorCitizen"] == 0) else 0, axis=1)

# Herhangi bir streaming hizmeti alan kişiler
df["NEW_FLAG_ANY_STREAMING"] = df.apply(lambda x: 1 if (x["StreamingTV"] == "Yes") or (x["StreamingMovies"] == "Yes") else 0, axis=1)

# Kişi otomatik ödeme yapıyor mu?
df["NEW_FLAG_AutoPayment"] = df["PaymentMethod"].apply(lambda x: 1 if x in ["Bank transfer (automatic)","Credit card (automatic)"] else 0)

# ortalama aylık ödeme
df["NEW_AVG_Charges"] = df["TotalCharges"] / (df["tenure"] + 1)

# Servis başına ücret
df["NEW_AVG_Service_Fee"] = df["MonthlyCharges"] / (df['NEW_TotalServices'] + 1)

#
df.loc[(df['InternetService'] == 'DSL') | (df['InternetService'] == "Fiber optic"), 'Internet_Service'] = 1
df.loc[df['InternetService'] == 'No', 'Internet_Service'] = 0

#Yaşlı/GEnç Kadın/Erkek Ayrımı
df.loc[(df['gender'] == 'Male') & (df['SeniorCitizen'] == 1), 'AgeGender'] = 'Senior_Male'
df.loc[(df['gender'] == 'Male') & (df['SeniorCitizen'] == 0) , 'AgeGender'] = 'Young_Male'
df.loc[(df['gender'] == 'Female') & (df['SeniorCitizen'] == 1), 'AgeGender'] = 'Senior_Female'
df.loc[(df['gender'] == 'Female') & (df['SeniorCitizen'] == 0), 'AgeGender'] = 'Young_Female'

df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TENURE_CAT_NEW,MONTHLY_CHARGES_NEW,Contract_Extra_Services,NEW_TotalServices,NEW_Engaged,NEW_noProt,NEW_Young_Not_Engaged,NEW_FLAG_ANY_STREAMING,NEW_FLAG_AutoPayment,NEW_AVG_Charges,NEW_AVG_Service_Fee,Internet_Service,AgeGender
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.850,0,Very_short,upper_eco,1,1,0,1,1,0,0,14.925,14.925,1.000,Young_Female
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,1889.500,0,long2,premium,3,3,1,1,0,0,0,53.986,14.238,1.000,Young_Male
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.150,1,Short,2x_upper_eco,3,3,0,1,1,0,0,36.050,13.463,1.000,Young_Male
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,1840.750,0,long2,2x_upper_eco,3,3,1,1,0,0,1,40.016,10.575,1.000,Young_Male
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.650,1,Short,platinium,1,1,0,1,1,0,0,50.550,35.350,1.000,Young_Female


In [25]:
# Yeni toplam ödemenin ekonomi premium gibi 
df['TotalCharges_New'] = pd.cut(x = df['TotalCharges'], bins = [-1, 500 ,2000, 4000, 9000],
                                  labels = [ 'economy', 'premium','prex2', 'prex3'])
target_summary_with_cat(df, 'Churn', 'TotalCharges_New')

df['Dependent_contracts'] = df['Dependents']+'_'+ df['Contract']

                  TARGET_MEAN
TotalCharges_New             
economy                 0.414
premium                 0.234
prex2                   0.238
prex3                   0.147




In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   gender                   7032 non-null   object  
 1   SeniorCitizen            7032 non-null   int64   
 2   Partner                  7032 non-null   object  
 3   Dependents               7032 non-null   object  
 4   tenure                   7032 non-null   int64   
 5   PhoneService             7032 non-null   object  
 6   MultipleLines            7032 non-null   object  
 7   InternetService          7032 non-null   object  
 8   OnlineSecurity           7032 non-null   object  
 9   OnlineBackup             7032 non-null   object  
 10  DeviceProtection         7032 non-null   object  
 11  TechSupport              7032 non-null   object  
 12  StreamingTV              7032 non-null   object  
 13  StreamingMovies          7032 non-null   object  
 14  Contract